In [13]:
from collections import namedtuple
import os
#os.chdir("../")

In [37]:
os.getcwd()

'd:\\Bala\\DEEP_CNN_CLASSIFIER'

In [38]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallBackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [39]:
from deepClassifier.constant import *
from deepClassifier.utils.common import read_yaml, create_directories

In [40]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_call_back_config(self) -> PrepareCallBackConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
            ])

        prepare_call_back_config = PrepareCallBackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
            
        )

        return prepare_call_back_config

In [51]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time



class PrepareCallBack:
    def __init__(self, config: PrepareCallBackConfig):
        self.config = config


    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log)
    
    @property
    def _create_ckpt_callbacks(self):
        filepath = self.config.checkpoint_model_filepath
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]   


In [52]:
try:
    config = ConfigurationManager()
    prepare_call_back_config = config.get_prepare_call_back_config()
    prepare_call_back = PrepareCallBack(config=prepare_call_back_config)
    prepare_call_back.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2024-06-20 12:39:38,475: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2024-06-20 12:39:38,478: INFO: common]: yaml file: params.yaml loaded successfully
[2024-06-20 12:39:38,480: INFO: common]: created directory at: artifacts
[2024-06-20 12:39:38,481: INFO: common]: created directory at: artifacts\prepare_callbacks\checkpoint_dir
[2024-06-20 12:39:38,482: INFO: common]: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir
